In [1]:
import os
import json
import datetime
from typing import *
from database.driver import MysqlDB
from database.init_db import init_DB

In [2]:
# 初始化NLQ数据库
init_DB()

execution success! NLQ database initialized!


True

In [3]:
# 导入数据（请先生成足够多e.g. 10000条数据）
resume_folder_path = "./temp/resume/"
db_obj = MysqlDB()

person_list = []
exp_list = []
label_list = []
for file in os.scandir(resume_folder_path):
    with open(file.path, 'r', encoding='utf-8') as f:
        cur_dict = json.load(f)
        # 读出人员信息
        person_list.append((str(cur_dict["id"]), str(cur_dict["id"]), cur_dict['教育经历'], int(cur_dict['年龄']), 1 if cur_dict['性别']=="男" else 0, cur_dict['职级']))
        # 读出经历信息
        for exp_idx,(exp_k,exp_v) in enumerate(cur_dict['工作经历'].items()):
            start_time = datetime.datetime.strptime(exp_v['start_time'], '%Y.%m').date()
            end_time = datetime.datetime.strptime(exp_v['end_time'], '%Y.%m').date()
            exp_list.append((str(cur_dict["id"])+'-'+str(exp_idx), exp_k, start_time, end_time, str(cur_dict["id"])))
            # 读出标签信息
            for lb_idx, lb in enumerate(exp_v["label"]):
                label_list.append((None, lb, str(cur_dict["id"]), (str(cur_dict["id"])+'-'+str(exp_idx)), start_time, end_time))

# 向数据库中插入人员信息
sql = "REPLACE INTO person (person_id, person_name, edu, age, gender, jobrank) VALUES (%s,%s,%s,%s,%s,%s)"
db_obj.execute_many(sql, [(p) for p in person_list])

# 向数据库中插入经历信息
sql = "REPLACE INTO experience (exp_id, exp_text, time_start, time_end, person_id) VALUES (%s,%s,%s,%s,%s)"
db_obj.execute_many(sql, [(e) for e in exp_list])

# 向数据库中插入标签信息
sql = "INSERT INTO label (label_id, label_text, person_id, exp_id, time_start, time_end) VALUES (%s,%s,%s,%s,%s,%s)"
db_obj.execute_many(sql, [(l) for l in label_list])


2217

In [4]:
# 查询示例（双标签，需要支持更多标签只需多JOIN几次）
label_a = ("教育", 2, "以上")
label_b = ("财务会计", 8, "以下")

condition_a = "%s %d"%(">=" if label_a[2]=="以上" else "<", label_a[1])
condition_b = "%s %d"%(">=" if label_b[2]=="以上" else "<", label_b[1])

sql = """SELECT a.person_id,a.label_text,TIMESTAMPDIFF(YEAR,a.time_start,a.time_end) duration_a,b.label_text,TIMESTAMPDIFF(YEAR,b.time_start,b.time_end) duration_b 
         FROM label a JOIN label b ON a.person_id=b.person_id
         WHERE a.label_text='{}' AND b.label_text='{}' AND TIMESTAMPDIFF(YEAR,a.time_start,a.time_end) {} AND TIMESTAMPDIFF(YEAR,a.time_start,a.time_end) {}""".format(label_a[0], label_b[0], condition_a, condition_b)

res = db_obj.query(sql)
print(res)

(('230', '教育', 4, '财务会计', 5), ('309', '教育', 4, '财务会计', 4), ('47', '教育', 7, '财务会计', 7), ('789', '教育', 4, '财务会计', 3), ('87', '教育', 4, '财务会计', 3))
